# Tears of the Kingdom Map Generation

All maps generated depend on the following parameters:

In [1]:
from PIL import Image

# Open the image file
map_image_path = "map_data/base_map_low_res.png"
map = Image.open(map_image_path)

# Get the width and height of the image
map_width_px, map_height_px = map.size

# Print the pixel dimensions
print("Width:", map_width_px)
print("Height:", map_height_px)

# Set the dpi you wish to use
dpi = 300

# calculate width and height in inches
map_width_in = map_width_px / dpi
map_height_in = map_height_px / dpi

map_extent = [0, map_width_in, 0, map_height_in]

Width: 3683
Height: 2996


The following code is used to convert between TOTK coordinates and coordinates on our folium map.

In [2]:
thyphio = [343,3133] # xy coordinates from TOTK
lindor = [-1901,1243] # xy coordinates from TOTK

thyphio_new = [676.9016,331.7492] # Where it should be placed on our folium map. 
lindor_new = [505.1637, 196.5511] # Where it should be placed on our folium map.


def scale_lat(y):
    # Note that the x-coordinate is latitude, so corresponds to the y-coordinates from TOTK.
    m = (thyphio_new[0]-lindor_new[0])/(thyphio[1]-lindor[1])
    return m*(y - lindor[1]) + lindor_new[0]
def scale_long(x):
    m = (thyphio_new[1]-lindor_new[1])/(thyphio[0]-lindor[0])
    return m*(x - lindor[0]) + lindor_new[1]

In [3]:
# # Code used to convert the high-res background into a lower resolution.

# from PIL import Image

# # Load the high-resolution photo
# image_path = 'map_data/base_map.png'  # Replace with the path to your high-resolution photo
# image = Image.open(image_path)

# # Define the desired percentage of the original size
# percentage = 40  # Replace with the desired percentage

# # Calculate the target width and height based on the percentage
# target_width = int(image.width * percentage / 100)
# target_height = int(image.height * percentage / 100)

# # Resize the image to the target resolution
# resized_image = image.resize((target_width, target_height), Image.LANCZOS)

# # Save the resized image to a file
# output_path = 'map_data/base_map_low_res.png'  # Replace with the path to save the resized photo
# resized_image.save(output_path)


# Sky Tower Data

In [4]:
import pandas as pd
import numpy as np
import re
import csv
# Read the lines from the file into a Pandas Series
skytowers = pd.read_csv('map_data/skyview_towers.csv')
skytowers['latitude'] = skytowers.y.apply(scale_lat)
skytowers['longitude'] = skytowers.x.apply(scale_long)
skytowers.head(5)

,tower,x,y,z,latitude,longitude
0,Lookout Landing Skyview Tower,-293,137,25,404.665225,293.431022
1,Hyrule Field Skyview Tower,753,-1019,64,299.623420,356.451170
2,Lindor’s Brow Skyview Tower,-1901,1243,297,505.163700,196.551100
3,Popla Foothills Skyview Tower,602,-2119,98,199.670145,347.353615
4,Sahasra Slope Skyview Tower,1344,-1170,166,285.902561,392.058156


Here, we create a map layer for sky towers.

In [5]:
import folium 

# Add the Sky Towers
tower_layer = folium.FeatureGroup(name='Sky Towers')
for index, row in skytowers.iterrows():
    custom_icon = folium.features.CustomIcon(icon_image='map_data/icons/towers.png',icon_size=(34,34))
    marker_x = folium.Marker(
        location=[row.latitude,row.longitude],  # Assuming x_scaled and y_scaled are in latitude and longitude format
        icon=custom_icon,
        tooltip="""<h3 class="zelda-popup">{}</h3> <br> <p class = "zelda-popup">x: {}, y: {}, z: {}</p>""".format(row['tower'],row['x'], row['y'],row['z'])
    )
    marker_x.add_to(tower_layer)

# Shrines

In [6]:
import re
shrines = pd.read_csv("map_data/shrines.csv")
# Here, we mark shrines by location. We will only include land shrines on our map.
shrines['area'] = ['sky' if x is not None else 'land' for x in shrines.Location.apply(lambda x: re.search("Sky",x))]
shrines['longitude'] = shrines.X.apply(scale_long)
shrines['latitude'] = shrines.Y.apply(scale_lat)
#[x for x in shrines.Location.uniq]
shrines.head(3)

,Shrine Name,Lightroot Name,Title,Type,Location,Region,Map Layer,In Cave?,Shrine Quest,X,...,Lightroot X,Lightroot Y,Lightroot Height,Shrine Name (Japanese),Lightroot Name (Japanese),ActorName,Lightroot ActorName,area,longitude,latitude
0,Ukouh Shrine,-,The Ability to Create,Puzzle,Great Sky Island,Central Hyrule Sky,Sky,No,-,275,...,-,-,-,ウコウホの祠,-,Dungeon060,-,sky,327.652288,309.255281
1,Gutanbac Shrine,-,The Ability to Rise,Puzzle,Great Sky Island,Central Hyrule Sky,Sky,No,-,709,...,-,-,-,グタンバチの祠,-,Dungeon061,-,sky,353.800227,266.547972
2,In-isa Shrine,-,The Ability to Combine,Puzzle,Great Sky Island,Central Hyrule Sky,Sky,No,-,26,...,-,-,-,インイサの祠,-,Dungeon062,-,sky,312.650360,255.553112


In [7]:
# Create a layer for the shrines
shrine_layer = folium.FeatureGroup(name='Shrines')
for index, row in shrines[shrines.area=='land'].iterrows():
    custom_icon = folium.features.CustomIcon(icon_image='map_data/icons/shrines.png',icon_size = (24,24))
    marker_x = folium.Marker(
        location = [row.latitude,row.longitude],
        icon = custom_icon,
        tooltip="""<h3 class = "zelda-popup">{}</h3> <br> <p class="zelda-popup">x: {}, y: {}, z: {}</p>""".format(row['Shrine Name'],row['X'], row['Y'],row['Height'])    )
    marker_x.add_to(shrine_layer)

# Korok Seeds
Here, we create a layer for korok seeds. First, we import the data.

In [8]:
korok_data = pd.read_csv("map_data/korok_seeds.csv")
korok_data['latitude'] = korok_data.Y.apply(scale_lat).apply(float) # Y value is latitude.
korok_data['longitude'] = korok_data.X.apply(scale_long).apply(float) # X value is longitude.

korok_data.head(5)

,ActorName,Type,X,Y,Height,Hash ID,latitude,longitude
0,KorokCarryProgressKeeper,Bring the exhausted korok to its friend,-4877,-3565,40,0xb9ed273015b98834,68.277021,17.250946
1,KorokCarryProgressKeeper,Bring the exhausted korok to its friend,-4508,-590,512,0xba0dbba90c1a9a19,338.605197,39.482719
2,KorokCarryProgressKeeper,Bring the exhausted korok to its friend,-4321,-1861,66,0x9c6ce54cb8aa0d77,223.113731,50.749227
3,KorokCarryProgressKeeper,Bring the exhausted korok to its friend,-4125,3431,223,0xc8de69e1d4d01529,703.979851,62.557974
4,KorokCarryProgressKeeper,Bring the exhausted korok to its friend,-4116,2416,-13,0x95684aa7a4c3b176,611.750238,63.100213


Next, we create a map layer for korok seeds:

In [9]:
"""<div class='zelda-popup'><p style="font-size:12px;">x: {}, y: {}, z: {}</p></div>""".format(row['X'], row['Y'],row['Height'])

'<div class=\'zelda-popup\'><p style="font-size:12px;">x: 931, y: -1903, z: 44.0</p></div>'

In [10]:
# Create a layer for the markers
korok_seeds = folium.FeatureGroup(name='Korok Seeds')
# Add markers for x_scaled and y_scaled
for index, row in korok_data.iterrows():
    if row['Height']<800:
        custom_icon = folium.features.CustomIcon(icon_image='map_data/icons/korok-seed.png',icon_size=(20,20))
        marker_x = folium.Marker(
            location=[row.latitude,row.longitude],  # Assuming x_scaled and y_scaled are in latitude and longitude format
            icon=custom_icon,
            popup="""<div class='zelda-popup'><p style="font-size:12px;">x: {}, y: {}, z: {}</p></div>""".format(row['X'], row['Y'],row['Height'])
        )
        marker_x.add_to(korok_seeds)
    else:
        custom_icon = folium.features.CustomIcon(icon_image='map_data/icons/korok-seed-sky.png',icon_size=(16,16))
        marker_x = folium.Marker(
            location=[row.latitude,row.longitude],  # Assuming x_scaled and y_scaled are in latitude and longitude format
            icon=custom_icon,
            popup="""<div class='zelda-popup'><p style="font-size:12px; ">x: {} <br> y: {} <br> z: {}</p></div>""".format(row['X'], row['Y'],row['Height'])
        )
        marker_x.add_to(korok_seeds)

# Dispensers

In [11]:
dispensers = pd.read_csv("map_data/dispensers.csv")

dispensers['latitude'] = dispensers.Y.apply(scale_lat)
dispensers['longitude'] = dispensers.X.apply(scale_long)

# Here, we create an HTML label containing information about the Zonai devices the dispenser provides.
dispensers['items'] = [f"""
<p class="zelda-popup" style="font-size: 16px; margin-bottom: 10px;">Location: {{}}</p>
<table class="zelda-popup zelda-table">
  <tr>
    <td>{{}}</td>
    <td>{{}}</td>
  </tr>
  <tr>
    <td>{{}}</td>
    <td>{{}}</td>
  </tr>
  <tr>
    <td>{{}}</td>
    <td>{{}}</td>
  </tr>
  <tr>
    <td>{{}}</td>
    <td>{{}}</td>
  </tr>
  <tr>
    <td>{{}}</td>
    <td>{{}}</td>
  </tr>
</table>
""".format(row['Map Layer'],row['Zonai Device 1'],row['Chance 1'],
          row['Zonai Device 2'],row['Chance 2'],
          row['Zonai Device 3'],row['Chance 3'],
          row['Zonai Device 4'],row['Chance 4'],
          row['Zonai Device 5'],row['Chance 5']) for index,row in dispensers.iterrows()]
dispensers.head(3)

,Name,Location,Map Layer,Zonai Device 1,Chance 1,Zonai Device 2,Chance 2,Zonai Device 3,Chance 3,Zonai Device 4,...,Device 2-1,Device 2-2,Device 2-3,Lottery Method 3,Device 3-1,Device 3-2,Device 3-3,latitude,longitude,items
0,Device Dispenser,Great Sky Island,Sky,Flame Emitter,20%,Fan,40%,Portable Pot,40%,-,...,Fan,Flame Emitter,Portable Pot,RandomExcludePrevDrop,Portable Pot,Fan,Flame Emitter,239.015388,337.292081,"\n<p class=""zelda-popup"" style=""font-size: 16p..."
1,Device Dispenser,Great Sky Island,Sky,Flame Emitter,15%,Wing,35%,Fan,30%,Portable Pot,...,Wing,Fan,-,-,-,-,-,277.724566,357.415149,"\n<p class=""zelda-popup"" style=""font-size: 16p..."
2,Device Dispenser,West Hebra Sky Archipelago,Sky,Rocket,30%,Flame Emitter,30%,Portable Pot,15%,Time Bomb,...,Flame Emitter,Rocket,Time Bomb,RandomExcludePrevDrop,Flame Emitter,Rocket,Time Bomb,643.644419,81.958058,"\n<p class=""zelda-popup"" style=""font-size: 16p..."


Create a layer for dispensers:

In [12]:
# create a layer for the dispensers
dispenser_layer = folium.FeatureGroup(name='Zonai Dispensers')
for index, row in dispensers.iterrows():
    custom_icon = folium.features.CustomIcon(icon_image='map_data/icons/dispenser.png',icon_size=(38,38))
    marker = folium.Marker(
        location = [row.latitude,row.longitude],
        icon=custom_icon,
        popup = row['items']
    )
    marker.add_to(dispenser_layer)

# The Map

In [39]:
import math
from folium import plugins

# Map is too big. Scaling here. Note that rescaling the map requires you to recalculate the scaling_lat and scaling_long functions above.
# You'll have to find the position of the skytowers in the new folium coordinates.
bounds = [[0, 0], [map_width_in*(1500/map_width_in)/2, map_height_in*(1500/map_width_in)/2]]
center_lat = (bounds[0][0] + bounds[1][0])/2
center_lon = (bounds[0][1] + bounds[1][1])/2 

# Calculate the zoom level based on the map width
zoom = 1  # Adjust this value based on your desired zoom level
# if map_width_in > 0:
#     zoom = int(13 - math.log2(map_width_in))

# Create the base map
base_map = folium.Map(
    location=[center_lat, center_lon],
    crs='Simple',
    zoom_start=zoom,
    tiles=None,
    max_bounds=True,
    min_lat = bounds[0][0],
    max_lat = bounds[1][0],
    min_lon = bounds[0][1],
    max_lon=bounds[1][1]
)

bounds

# Add the background image.
totk_overlay = folium.raster_layers.ImageOverlay(
    image = 'map_data/base_map_low_res.png',
    bounds=bounds,
    overlay=False,
    control=False,
    name="TOTK Map",
    zindex=1
)

totk_overlay.add_to(base_map)


# Ad the sky towers layer
tower_layer.add_to(base_map)
# Add korok seeds layer
korok_seeds.add_to(base_map)
# Add shrine layer
shrine_layer.add_to(base_map)
# Add dispensers
dispenser_layer.add_to(base_map)

#plugins.Fullscreen().add_to(base_map)
base_map.add_child(plugins.Fullscreen())
#base_map.fit_bounds(bounds=marker_layer.get_bounds())

# Add layer control to the map
folium.LayerControl().add_to(base_map)

# # This next line is useful for when you have to rescale and locate lat/long on the map. Disable if you need.
lat_lng_popup = folium.LatLngPopup()
lat_lng_popup.add_to(base_map)


# Define custom CSS styling for the popup
custom_style = """
<style>
    .leaflet-popup-content-wrapper {
        background-color: #0b6623 !important;
        border-radius: 5px !important;
        box-shadow: 0 2px 10px rgba(0, 0, 0, 0.2) !important;
    }
    
    .leaflet-popup-content {
        color: #ffd700 !important;
        font-family: 'Arial', sans-serif !important;
        font-size: 16px !important;
        text-align: center !important;
    }

    .zelda-table {
    border-collapse: collapse;
    width: 100%;
    }
    
    .zelda-table td {
    border: 1px solid gold;
    padding: 5px;
    }
    
    .leaflet-popup-tip-container,
    .leaflet-tooltip-tip {
        display: none !important;
    }
    
    .leaflet-tooltip {
        background-color: #0b6623 !important;
        color: #ffd700 !important;
        font-family: 'Arial', sans-serif !important;
        font-size: 16px !important;
        text-align: center !important;
        border: none !important;
        box-shadow: none !important;
    }
    
    .zelda-popup h3 {
        margin: 0 !important;
        padding: 5px !important;
        font-size: 20px !important;
        font-weight: bold !important;
        color: #ffd700 !important;
        text-shadow: none !important;
    }
</style>
"""


# Add the custom CSS styling to the map
base_map.get_root().header.add_child(folium.Element(custom_style))

base_map.save('totk-map.html')
    
# base_map